In [ ]:
# Preparando kernel
import numpy as np
import pandas as pd
!pip3 install pickle5
import pickle5 as pickl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-linux_x86_64.whl size=256429 sha256=6affb770fb61b869d269c011ae2b8757b92f84a4dd3058705cff81ed53eb84e2
  Stored in directory: /root/.cache/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5


In [ ]:
# Montando o Google drive
from google.colab import drive
drive.mount('/content/drive')
# Para ler/carregar
#path = '/content/drive/MyDrive/Operações 3/'
path = '/content/drive/MyDrive/Classroom/OpUnit3_files/OU3_Python/'
with open(path + 'databank_properties.pickle', 'rb') as handle:
    dados, all_units = pickl.load(handle)
print(dados.shape)
# Incluindo uma pasta do drive (mounted) no path do COLAB
import sys
sys.path.append(path)
# Carregamento das funções úteis
from OU3_funcoes_uteis_v05_2 import *

Mounted at /content/drive


In [ ]:
# Para procurar o 'num' de um determinado componente pelo nome
dados[dados.name.str.contains('water', case = False)]

,num,name,molar_mass,fusion_point,boiling_point,critical_temp,critical_pressure,critical_volume,zc,acentric_factor,...,pvap_a,pvap_b,pvap_c,temp_max_pvap,temp_min_pvap,pvap_h_a,pvap_h_b,pvap_h_c,pvap_h_d,delta_h_vap_bp
466,20,WATER,18.015,273.2,373.2,647.3,217.6,56.0,0.229,0.344,...,18.3036,3816.44,-46.13,441.0,284.0,55.336,-6869.5,-5.115,1.05,9717.0


In [ ]:
# Seleção dos componentes presentes no sistema
lista_componentes = [122, 62, 20]
nc = len(lista_componentes)
nc

3

In [ ]:
dados[dados['num'].isin(lista_componentes)][['num','name']]

,num,name
145,122,ACETONE
296,62,METHANOL
466,20,WATER


In [ ]:
# Verificando os pontos de ebulição de nossos componentes
T_eb_comp = dados[dados['num'].isin(lista_componentes)]['boiling_point']
T_eb_comp = T_eb_comp.tolist()
T_eb_comp

[329.4, 337.8, 373.2]

In [ ]:
# Valores do a_W - Tabela 12.5, p. 354 SVNA
# acetona = 1
# metanol = 2
# água    = 3
nc = 3
a_W = np.zeros((nc,nc))
# acetona e água
a_W[1-1,3-1] = 291.27
a_W[3-1,1-1] = 1448.01
# metanol e água
a_W[2-1,3-1] = 107.38
a_W[3-1,2-1] = 469.55
# acetona e metanol
a_W[1-1,2-1] = -161.88
a_W[2-1,1-1] = 583.11
#
a_W

array([[   0.  , -161.88,  291.27],
       [ 583.11,    0.  ,  107.38],
       [1448.01,  469.55,    0.  ]])

In [ ]:
V_m = f_Vol_molar_db(lista_componentes, dados)
V_m

array([73.51898734, 40.50821745, 18.0511022 ])

In [ ]:
z = np.array([0.1,0.2,0.7])

In [ ]:
P_eq = 101.33*(760.0/101.325) # mmHg
P_eq

760.0375030841352

In [ ]:
def f_calc_T_Pb_nid_Wilson_01_db(z, P_eq, a_W, V_m, lista_componentes, dados):
  ''' Algoritmo de BOL_T descrito na figura 14.3 p.409 do SVNA
  '''
  nc = z.shape[0]
  resp = f_calculo_PbPo_db('T', P_eq, z, lista_componentes, dados)
  T_pb_ideal = resp[0]
  T_pb = T_pb_ideal
  x_eq = z
  y_eq = f_sol_RR_flash_db(z, P_eq, T_pb, lista_componentes, dados)['y_eq']
  K_comp = f_K_Raoult_mod_Wilson_01_db(T_pb, P_eq, x_eq, y_eq, a_W, V_m, 
                                       lista_componentes, dados)
  # Eq.14.8 do SVNA
  y_eq_novo = K_comp * x_eq
  phi_comp = f_phi_gen(y_eq_novo, T_pb, P_eq)
  gama_comp = f_gama_Wilson(x_eq, T_pb, a_W, V_m)
  # Componente de referência
  jj = 2
  # Eq. 14.13 do SVNA
  fator = x_eq*gama_comp/phi_comp
  Pvap_comp = f_Pvap_Antoine_vetor_db(T_pb, lista_componentes, dados)
  razao = Pvap_comp/Pvap_comp[jj]
  somat = fator @ razao
  Pvap_novo = P_eq / somat
  # Eq. 14.15 do SVNA
  T_pb_novo = f_Teq_Antoine_db(Pvap_novo, lista_componentes[jj], dados)[0]
  erro_T = T_pb_novo - T_pb
  # Tolerância para convergência
  epsilon = 1.0e-3
  #
  T_pb = T_pb_novo
  y_eq = y_eq_novo
  k_iter = 1
  n_max_iter = 20
  while ((np.abs(erro_T).mean() > epsilon)&(k_iter < n_max_iter)):
    #print(k_iter, np.abs(erro_T).mean())
    K_comp = f_K_Raoult_mod_Wilson_01_db(T_pb, P_eq, x_eq, y_eq, 
                                       a_W, V_m, lista_componentes, dados)
    y_eq_novo = K_comp * x_eq
    phi_comp = f_phi_gen(y_eq_novo, T_pb, P_eq)
    gama_comp = f_gama_Wilson(x_eq, T_pb, a_W, V_m)
    fator = x_eq*gama_comp/phi_comp
    Pvap_comp = f_Pvap_Antoine_vetor_db(T_pb, lista_componentes, dados)
    T_pb_novo = np.zeros((nc,))
    for j in range(0,nc):
      razao = Pvap_comp/Pvap_comp[j]
      somat = fator @ razao
      Pvap_novo = P_eq / somat
      T_pb_novo[j] = f_Teq_Antoine_db(Pvap_novo, lista_componentes[j], dados)[0]
    erro_T = T_pb_novo[jj] - T_pb
    T_pb = T_pb_novo[jj] # T_pb_novo.mean()
    y_eq = y_eq_novo
    k_iter += 1
  #
  return {'T_pb': T_pb, 'y_eq':y_eq, 'erro_T': np.abs(erro_T)}

In [ ]:
sol_T_Pb = f_calc_T_Pb_nid_Wilson_01_db(z, P_eq, a_W, V_m, lista_componentes, dados)
sol_T_Pb

{'T_pb': 354.89790734030686,
 'y_eq': array([0.22367947, 0.42552852, 0.35080331]),
 'erro_T': 0.0002821873965785926}

In [ ]:
#caminho = '/content/drive/MyDrive/PFG_Mateus_Ferreira/'
caminho = '/content/drive/MyDrive/PFG_EQ/PFG_Mateus_Ferreira/'
arquivo_excel = 'Dados_validacao_Bolha_T_Wilson.xlsx'
df = pd.read_excel(caminho+arquivo_excel)
df.head()

,T [K],x1 [mol/mol],x2 [mol/mol],y1 [mol/mol],y2 [mol/mol]
0,345.95,0.0188,0.3778,0.0912,0.6450
1,340.65,0.0253,0.6722,0.0773,0.7869
2,342.35,0.0331,0.5564,0.0953,0.7191
3,337.35,0.0429,0.8710,0.0945,0.8710
4,337.55,0.0449,0.8439,0.1018,0.8496


In [ ]:
#Tamanho do Data Frame:
df.shape

(54, 5)

In [ ]:
#Trocando o nome das colunas para facilitar:
#
df['x1'] = df.pop('x1\xa0[mol/mol]')
df['x2'] = df.pop('x2\xa0[mol/mol]')
df['y1'] = df.pop('y1\xa0[mol/mol]')
df['y2'] = df.pop('y2\xa0[mol/mol]')  
#
df.head()

,T [K],x1,x2,y1,y2
0,345.95,0.0188,0.3778,0.0912,0.6450
1,340.65,0.0253,0.6722,0.0773,0.7869
2,342.35,0.0331,0.5564,0.0953,0.7191
3,337.35,0.0429,0.8710,0.0945,0.8710
4,337.55,0.0449,0.8439,0.1018,0.8496


In [ ]:
#Inserindo as colunas com os valores de x3 e y3
df['x3'] =  1 - (df['x1'] + df['x2'])
df['y3'] =  1 - (df['y1'] + df['y2'])
#
#Reordenando as colunas do Data Frame
df = df[['T [K]','x1', 'x2', 'x3', 'y1', 'y2', 'y3']]
df.head()

,T [K],x1,x2,x3,y1,y2,y3
0,345.95,0.0188,0.3778,0.6034,0.0912,0.6450,0.2638
1,340.65,0.0253,0.6722,0.3025,0.0773,0.7869,0.1358
2,342.35,0.0331,0.5564,0.4105,0.0953,0.7191,0.1856
3,337.35,0.0429,0.8710,0.0861,0.0945,0.8710,0.0345
4,337.55,0.0449,0.8439,0.1112,0.1018,0.8496,0.0486


In [ ]:
par = np.array([291.27, 1448.01, 107.38, 469.55, -161.88, 583.11])
#par = np.array([350.0, 1448.01, 107.38, 469.55, -161.88, 583.11])
par

array([ 291.27, 1448.01,  107.38,  469.55, -161.88,  583.11])

In [ ]:
df.loc[0,'T [K]']

345.95

In [ ]:
def f_res_par(par, P_eq, V_m, lista_componentes, dados, df):
  '''
    Entradas:
      df: dataframe com os dados experimentais para a estimação
    Saidas:

  '''
  nc = 3
  a_W = np.zeros((nc,nc))
  #
  a_W[1-1,3-1] = par[0]
  a_W[3-1,1-1] = par[1]
  # metanol e água
  a_W[2-1,3-1] = par[2]
  a_W[3-1,2-1] = par[3]
  # acetona e metanol
  a_W[1-1,2-1] = par[4]
  a_W[2-1,1-1] = par[5]
  #
  composicoes = df[['x1', 'x2', 'x3']]
  x1 = composicoes['x1'].tolist()
  x2 = composicoes['x2'].tolist()
  x3 = composicoes['x3'].tolist()
  z = np.array(list(zip(x1, x2, x3)))
  #
  n_exp = df.shape[0]
  res_T = np.zeros((n_exp,))
  res_y1 = np.zeros((n_exp,))
  res_y2 = np.zeros((n_exp,))
  res_y3 = np.zeros((n_exp,))
  for k in range(0,n_exp):
    #print(k)
    sol_T_Pb = f_calc_T_Pb_nid_Wilson_01_db(z[k], P_eq, a_W, V_m, lista_componentes, dados)
    res_T[k] = df.loc[k,'T [K]'] - sol_T_Pb['T_pb']
    res_y1[k] = df.loc[k,'y1'] - sol_T_Pb['y_eq'][0]
    res_y2[k] = df.loc[k,'y2'] - sol_T_Pb['y_eq'][1]
    res_y3[k] = df.loc[k,'y3'] - sol_T_Pb['y_eq'][2]
  #
  #res = np.zeros((n_exp,))
  res = np.abs(res_T) + np.abs(res_y1) + np.abs(res_y2) + np.abs(res_y3)
  return (res)

In [ ]:
%%capture
res = f_res_par(par, P_eq, V_m, lista_componentes, dados, df)

In [ ]:
res

array([ 5.40100362,  2.04683648,  2.89058428,  1.36292153,  1.5409993 ,
        9.33024106, 15.72440605,  4.90739831,  2.73263708, 22.45775259,
        7.3360775 , 13.72728775, 10.85460136,  4.1580424 ,  3.33549016,
        8.39386441,  7.217449  , 11.59738978,  4.2707586 ,  7.7210262 ,
        6.56119305,  8.37420861, 12.25158374,  4.61145903, 16.88414534,
        5.31518359,  9.75081963,  5.36650958,  7.09456823,  4.85603217,
        6.95420904, 11.05134783,  5.05355634,  5.45938183,  4.6815615 ,
        5.76803463,  9.6028513 ,  6.34611297,  8.94332439,  4.97515653,
        8.05434871,  7.06286126,  6.88444876,  4.17582948,  4.50786875,
        3.93556684,  3.98051762,  3.58208537,  2.74454912,  2.62231013,
        1.76468957,  1.35929745,  1.52341094,  0.57394589])

In [ ]:
par0= np.array([2552.62415561, -542.90749412, 714.24545824, -147.86284985, 291.73645055, 266.52211273])

In [ ]:
%%capture
from scipy.optimize import least_squares
# resp = least_squares(f_res_par, par, args=(P_eq, V_m, lista_componentes, dados, df))
resp = least_squares(f_res_par, par, method = 'lm',
                     args=(P_eq, V_m, lista_componentes, dados, df))

/usr/local/lib/python3.9/dist-packages/scipy/optimize/_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/usr/local/lib/python3.9/dist-packages/scipy/optimize/_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/usr/local/lib/python3.9/dist-packages/scipy/optimize/_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/usr/local/lib/python3.9/dist-packages/scipy/optimize/_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/usr/local/lib/python3.9/dist-packages/scipy/optimize/_minpack_p

`xtol` termination condition is satisfied.
Function evaluations 19, initial cost 3.2533e+00, final cost 3.2533e+00, first-order optimality 3.77e-04.


In [ ]:
resp

     message: `xtol` termination condition is satisfied.
     success: True
      status: 3
         fun: [ 8.522e-02  9.227e-02 ...  4.584e-01  1.645e-01]
           x: [ 2.553e+03 -5.429e+02  7.142e+02 -1.479e+02  2.917e+02
                2.665e+02]
        cost: 3.253326367354533
         jac: [[-3.161e-04 -1.367e-03 ... -3.199e-03 -6.966e-04]
               [ 5.397e-05  4.715e-04 ...  2.289e-03  7.225e-04]
               ...
               [-1.850e-04 -2.149e-03 ... -2.893e-03 -2.019e-03]
               [ 1.339e-04  1.566e-03 ...  2.526e-03  1.850e-03]]
        grad: [ 1.507e-04  3.496e-04  3.768e-04  3.663e-04  4.861e-05
               -4.424e-05]
  optimality: 0.00037682563920068995
 active_mask: [0 0 0 0 0 0]
        nfev: 19
        njev: None

 message: Both `ftol` and `xtol` termination conditions are satisfied.

     success: True

      status: 4

         fun: [ 8.522e-02  9.227e-02 ...  4.584e-01  1.645e-01]
           x: [ 2.553e+03 -5.429e+02  7.142e+02 -1.479e+02  2.917e+02
                2.665e+02]

        cost: 3.253326367348371

         jac: [[-3.161e-04 -1.367e-03 ... -3.199e-03 -6.966e-04]
               [ 5.398e-05  4.715e-04 ...  2.289e-03  7.225e-04]
               ...
               [-1.850e-04 -2.149e-03 ... -2.893e-03 -2.019e-03]
               [ 1.339e-04  1.566e-03 ...  2.526e-03  1.850e-03]]

        grad: [ 3.821e-04  1.167e-03  9.886e-04  7.758e-04  1.041e-03
                1.477e-04]

  optimality: 0.0011666829789313262

 active_mask: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00
                0.000e+00]

        nfev: 37

        njev: 23


'lm'

message: `xtol` termination condition is satisfied.
     success: True
      status: 3
         fun: [ 8.522e-02  9.227e-02 ...  4.584e-01  1.645e-01]
           x: [ 2.553e+03 -5.429e+02  7.142e+02 -1.479e+02  2.917e+02
                2.665e+02]
        cost: 3.253326367354533
         jac: [[-3.161e-04 -1.367e-03 ... -3.199e-03 -6.966e-04]
               [ 5.397e-05  4.715e-04 ...  2.289e-03  7.225e-04]
               ...
               [-1.850e-04 -2.149e-03 ... -2.893e-03 -2.019e-03]
               [ 1.339e-04  1.566e-03 ...  2.526e-03  1.850e-03]]
        grad: [ 1.507e-04  3.496e-04  3.768e-04  3.663e-04  4.861e-05
               -4.424e-05]
  optimality: 0.00037682563920068995
 active_mask: [0 0 0 0 0 0]
        nfev: 19
        njev: None

In [ ]:
par

array([ 291.27, 1448.01,  107.38,  469.55, -161.88,  583.11])

In [ ]:
par_opt = resp.x
par_opt

array([2552.62415561, -542.90749412,  714.24545824, -147.86284985,
        291.73645055,  266.52211273])

In [ ]:
%%capture
res = f_res_par(par_opt, P_eq, V_m, lista_componentes, dados, df)

method: 'trf'

par_opt = np.array([2552.62415561, -542.90749412,  714.24545824, -147.86284985, 291.73645055,  266.52211273])

par_opt = np.array([2552.62415561, -542.90749412,  714.24545824, -147.86284985, 291.73645055,  266.52211273])

In [ ]:
res

array([0.08522352, 0.09226557, 0.62031286, 0.13199824, 0.16161526,
       0.61747075, 0.34213842, 0.19952137, 0.12904121, 0.1168661 ,
       0.09950934, 0.10186069, 0.18544306, 0.49912888, 0.55300228,
       0.37803519, 0.27424735, 0.20150743, 0.65564844, 0.27439753,
       0.17428559, 0.16158753, 0.2250851 , 0.38675722, 0.40207148,
       0.16709863, 0.29871923, 0.05668015, 0.24365577, 0.50554464,
       0.88521332, 0.12581647, 0.22747176, 0.54922359, 0.13661605,
       0.81917127, 0.24556157, 0.27849906, 0.14541849, 0.42964105,
       0.40150098, 0.32271758, 0.53019912, 0.22261081, 0.13251644,
       0.08221887, 0.04445954, 0.34401981, 0.18237573, 0.15893626,
       0.03965697, 0.20685476, 0.45842814, 0.16450046])